**Resources**
1.   [Multilabel Classification](https://github.com/ashrefm/multi-label-soft-f1/blob/master/Multi-Label%20Image%20Classification%20in%20TensorFlow%202.0.ipynb)
2.   [Preprocessing Layers](https://www.tensorflow.org/tutorials/structured_data/preprocessing_layers)



In [ ]:
import pandas as pd
import tensorflow as tf
import gdown
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from tensorflow.keras import layers

In [ ]:
# Preparing the pandas dataframe

id = "1PuFzxZ7MykxfG7DWdZwfyIJ9E3elaQTt"
output = "datadummy_grouped.csv"
gdown.download(id=id, output=output, quiet=False)

df = pd.read_csv("/content/datadummy_grouped.csv")
df.head()

Downloading...
From: https://drive.google.com/uc?id=1PuFzxZ7MykxfG7DWdZwfyIJ9E3elaQTt
To: /content/datadummy_grouped.csv
100%|██████████| 136k/136k [00:00<00:00, 58.1MB/s]


,Unnamed: 0,Topics,Sub Topic,Type,Difficulty,Project Type,Workers
0,0,Back End,Django,S1,22,Backend,"['Chairul Rizqi', 'Abdullah Nur Hudi', 'Sandri..."
1,1,Data Engineering,Data Integration,S2,10,ML,"['Nyoman Satiya Najwa Sadha', 'Suci Rahmadani'..."
2,2,Time-series,LSTM,S2,15,ML,"['Putu Gede Agung Karna Sampalan', 'Rikip Gina..."
3,3,Computer Vision,Object Detection,S1,12,ML,"['Rikip Ginanjar', 'Farel Eden', 'Bagja Kurnia..."
4,4,NLP,Sentiment Analysis,S2,13,ML,"['Alvin Tan', 'Nyoman Satiya Najwa Sadha', 'Pu..."


In [ ]:
# Cleaning the unused columns
df = df.drop(df.columns[[0,3]], axis=1)

# Transform 'Workers' from strings into lists
df['Workers'] = df['Workers'].str.replace("[\'\[\]]","",regex=True)
df['Workers'] = df['Workers'].str.replace(", ","|",regex=True)
df['Workers'] = df['Workers'].apply(lambda s: [l for l in str(s).split('|')])
df.head()

,Topics,Sub Topic,Difficulty,Project Type,Workers
0,Back End,Django,22,Backend,"[Chairul Rizqi, Abdullah Nur Hudi, Sandrian Yu..."
1,Data Engineering,Data Integration,10,ML,"[Nyoman Satiya Najwa Sadha, Suci Rahmadani, Az..."
2,Time-series,LSTM,15,ML,"[Putu Gede Agung Karna Sampalan, Rikip Ginanja..."
3,Computer Vision,Object Detection,12,ML,"[Rikip Ginanjar, Farel Eden, Bagja Kurniadi, S..."
4,NLP,Sentiment Analysis,13,ML,"[Alvin Tan, Nyoman Satiya Najwa Sadha, Putu Ge..."


In [ ]:
# Creating list of labels
labels_list = df['Workers']
labels_list = list(labels_list)
mlb = MultiLabelBinarizer()
mlb.fit(labels_list)

N_LABELS = len(mlb.classes_)
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i, label))

0. Abdullah Nur Hudi
1. Abiyyu Diora Haqi
2. Alvin Tan
3. Andhika Zulfikri
4. Andi Rezal Oktavianto
5. Azis Sofyanto
6. Bagja Kurniadi
7. Chairul Rizqi
8. Christopher Kristianto
9. Farel Eden
10. Gabriel Kheisa
11. I Putu Ranantha Nugraha Suparta
12. Iga Narendra Pramawijaya
13. Imam
14. Muhammad Raden Syawali Akbar
15. Nyoman Satiya Najwa Sadha
16. Putu Gede Agung Karna Sampalan
17. Rikip Ginanjar
18. Sandrian Yulianto
19. Sarah Sema Khairunisa
20. Suci Rahmadani
21. Vania Kylie
22. Wahyu Fauzan


In [ ]:
df['Workers']

0      [Chairul Rizqi, Abdullah Nur Hudi, Sandrian Yu...
1      [Nyoman Satiya Najwa Sadha, Suci Rahmadani, Az...
2      [Putu Gede Agung Karna Sampalan, Rikip Ginanja...
3      [Rikip Ginanjar, Farel Eden, Bagja Kurniadi, S...
4      [Alvin Tan, Nyoman Satiya Najwa Sadha, Putu Ge...
                             ...                        
995    [Farel Eden, I Putu Ranantha Nugraha Suparta, ...
996    [Farel Eden, Christopher Kristianto, Putu Gede...
997    [Imam, Iga Narendra Pramawijaya, Muhammad Rade...
998    [Gabriel Kheisa, Andhika Zulfikri, Rikip Ginan...
999    [Muhammad Raden Syawali Akbar, Andi Rezal Okta...
Name: Workers, Length: 1000, dtype: object

In [ ]:
# Split the dataframe

# Daffa ganti ke 0.2 bentar

train, test = train_test_split(df, test_size=0.1)
train, val = train_test_split(train, test_size=0.1)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

810 train examples
90 validation examples
100 test examples


In [ ]:
# Input panda dataframe into tf.data pipeline

mlb = MultiLabelBinarizer()
mlb.fit(df['Workers'])

def df_to_dataset(dataframe, shuffle=True, batch_size=40):
  df = dataframe.copy() # copying the dataframe
  labels = df.pop('Workers') # separating the labels

  # Fit the multi-label binarizer on the training labels
  labels = list(labels)
  labels_bin = mlb.transform(labels).tolist()
  #labels_bin = pd.DataFrame(labels_bin, columns = ['Workers'])
  
  df = {key: value[:,tf.newaxis] for key, value in df.items()} # x
  ds = tf.data.Dataset.from_tensor_slices((dict(df), labels_bin))
  
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  ds = ds.prefetch(batch_size)
  return ds

In [ ]:
# Numerical Columns

def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
  normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
  normalizer.adapt(feature_ds)

  return normalizer

In [ ]:
# Categorical Columns

def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
  # Create a layer that turns strings into integer indices.
  if dtype == 'string':
    index = layers.StringLookup(max_tokens=max_tokens)
  # Otherwise, create a layer that turns integer values into integer indices.
  else:
    index = layers.IntegerLookup(max_tokens=max_tokens)

  # Prepare a `tf.data.Dataset` that only yields the feature.
  feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the set of possible values and assign them a fixed integer index.
  index.adapt(feature_ds)

  # Encode the integer indices.
  encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())

  # Apply multi-hot encoding to the indices. The lambda function captures the
  # layer, so you can use them, or include them in the Keras Functional model later.
  return lambda feature: encoder(index(feature))

In [ ]:
# Split and preprocess the dataframe

batch_size = 10
train_ds = df_to_dataset(train, shuffle=True, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

<ipython-input-7-ff026ce6b2cd>:15: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:,tf.newaxis] for key, value in df.items()} # x


In [ ]:
all_inputs = []
encoded_features = []

# Numerical features


numeric_cols = ['Difficulty']

for header in numeric_cols:
  numeric_col = tf.keras.Input(shape=(1,), name=header)
  normalization_layer = get_normalization_layer(header, train_ds)
  encoded_numeric_col = normalization_layer(numeric_col)
  all_inputs.append(numeric_col)
  encoded_features.append(encoded_numeric_col)

# Categorical features

categorical_cols = ['Topics', 'Sub Topic', 'Project Type']

for header in categorical_cols:
  categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='string')
  encoding_layer = get_category_encoding_layer(name=header,
                                               dataset=train_ds,
                                               dtype='string')
  encoded_categorical_col = encoding_layer(categorical_col)
  all_inputs.append(categorical_col)
  encoded_features.append(encoded_categorical_col)




In [ ]:
#Encode the Validation data and Test Data



In [ ]:
# Creating the model

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(42, activation="relu")(all_features)
x = tf.keras.layers.Dense(35, activation='softmax')(x)
x = tf.keras.layers.Dense(128, activation='softmax')(x)
output = tf.keras.layers.Dense(N_LABELS, activation='softmax')(x)

model = tf.keras.Model(all_inputs, output)



ValueError: ignored

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=["accuracy"])

In [ ]:
model.evaluate()

ValueError: ignored

In [ ]:
"""all_features = tf.keras.layers.concatenate(encoded_features)

model = tf.keras.Sequential([
    all_features,
    layers.Dense(100, activation='relu'),
    layers.Dense(50, activation='relu'),
    layers.Dense(N_LABELS, activation='linear')
    
])
model.summary"""

"all_features = tf.keras.layers.concatenate(encoded_features)\n\nmodel = tf.keras.Sequential([\n    all_features,\n    layers.Dense(100, activation='relu'),\n    layers.Dense(50, activation='relu'),\n    layers.Dense(N_LABELS, activation='linear')\n    \n])\nmodel.summary"

In [ ]:
# Training the model

model.fit(train_ds, epochs=100, validation_data=val_ds)

Epoch 1/100
81/81 [==============================] - 0s 3ms/step - loss: 11.3889 - accuracy: 0.0074 - val_loss: 12.2531 - val_accuracy: 0.0000e+00
Epoch 2/100
81/81 [==============================] - 0s 3ms/step - loss: 11.3920 - accuracy: 0.0074 - val_loss: 12.2581 - val_accuracy: 0.0000e+00
Epoch 3/100
81/81 [==============================] - 0s 3ms/step - loss: 11.3952 - accuracy: 0.0074 - val_loss: 12.2591 - val_accuracy: 0.0000e+00
Epoch 4/100
81/81 [==============================] - 0s 3ms/step - loss: 11.3987 - accuracy: 0.0074 - val_loss: 12.2649 - val_accuracy: 0.0000e+00
Epoch 5/100
81/81 [==============================] - 0s 3ms/step - loss: 11.4015 - accuracy: 0.0074 - val_loss: 12.2679 - val_accuracy: 0.0000e+00
Epoch 6/100
81/81 [==============================] - 0s 3ms/step - loss: 11.4023 - accuracy: 0.0074 - val_loss: 12.2682 - val_accuracy: 0.0000e+00
Epoch 7/100
81/81 [==============================] - 0s 3ms/step - loss: 11.4068 - accuracy: 0.0074 - val_loss: 12.273

In [ ]:
# Evaluate the model's accuracy

loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

10/10 [==============================] - 0s 2ms/step - loss: nan - accuracy: 0.1200
Accuracy 0.11999999731779099
